In [1]:
import pandas as pd
from pandas import DataFrame
from typing import List, Dict
import logging
import time
from functools import reduce
import requests
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects, RetryError
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

## All Coins List

In [6]:
def get_coins_list() -> List[dict]:
    return requests.get('https://api.coingecko.com/api/v3/coins/list').json()


coins_list = get_coins_list()
coins_list[:3]

[{'id': '01coin', 'symbol': 'zoc', 'name': '01coin'},
 {'id': '0-5x-long-algorand-token',
  'symbol': 'algohalf',
  'name': '0.5X Long Algorand Token'},
 {'id': '0-5x-long-altcoin-index-token',
  'symbol': 'althalf',
  'name': '0.5X Long Altcoin Index Token'}]

In [7]:
def get_coin_details(id: str, retries: int = 5) -> dict:
    for _ in range(retries):
        response = requests.get(f'https://api.coingecko.com/api/v3/coins/{id}')
        logging.debug(response.status_code)
        if response.status_code == 200: return response.json()
        else:
            time.sleep(1)
    return {}

In [8]:
def get_all_coins_details(coin_list: list, rate: int = 50) -> list:
    out_list = []
    request_pool = [0]
    while (len(coin_list) > 0):
        current_id = coin_list[0]["id"]
        request_pool.append(time.perf_counter())
        response = requests.get(f'https://api.coingecko.com/api/v3/coins/{current_id}')
        if len(request_pool) > rate:
            request_pool.pop(0)
        if response.status_code == 200:
            coin_list.pop(0)
            out_list.append(response.json())
            print(f"{current_id} ok, {len(coin_list)} more to go")
        else:
            requests_time_span = request_pool[-1] - request_pool[0]
            print(f"{requests_time_span + 1}\n{request_pool}")
            if (requests_time_span + 1) < rate:
                time.sleep(int(requests_time_span))
    return tuple(out_list)
            
        

In [10]:
all_reports = get_all_coins_details(coins_list[:200])

01coin ok, 199 more to go
0-5x-long-algorand-token ok, 198 more to go
0-5x-long-altcoin-index-token ok, 197 more to go
0-5x-long-ascendex-token-token ok, 196 more to go
0-5x-long-bitcoin-cash-token ok, 195 more to go
0-5x-long-bitcoin-token ok, 194 more to go
0-5x-long-cardano-token ok, 193 more to go
0-5x-long-chainlink-token ok, 192 more to go
0-5x-long-cosmos-token ok, 191 more to go
0-5x-long-dogecoin-token ok, 190 more to go
0-5x-long-eos-token ok, 189 more to go
0-5x-long-ethereum-classic-token ok, 188 more to go
0-5x-long-ethereum-token ok, 187 more to go
0-5x-long-kyber-network-token ok, 186 more to go
0-5x-long-matic-token ok, 185 more to go
0-5x-long-privacy-index-token ok, 184 more to go
0-5x-long-swipe-token ok, 183 more to go
0-5x-long-theta-network-token ok, 182 more to go
0-5x-long-xrp-token ok, 181 more to go
0cash ok, 180 more to go
0chain ok, 179 more to go
0-mee ok, 178 more to go
0x ok, 177 more to go
0xcert ok, 176 more to go
0xdao ok, 175 more to go
0xdao-v2 ok, 1

In [ ]:
coin_detail0 = get_coin_details(coins_list[0]["id"])
coin_detail1 = get_coin_details(coins_list[1]["id"])
coin_detail0

In [ ]:
pd.DataFrame([get_coin_details(coins_list[50]["id"])])

## Extract Dict Data Frame

In [ ]:
def get_dict_info(coin_detail: dict, column_name: str) -> dict:
    try: dict_info: dict = coin_detail[f"{column_name}"]
    except: return {"id": [coin_detail["id"]]}
    dict_info["id"] = coin_detail["id"]
    final_dict: dict = {key: [value] for key, value in dict_info.items()}
    if '' in final_dict.keys(): final_dict.pop('')
    return final_dict


dict_info0 = get_dict_info(coin_detail0, "platforms")
dict_info1 = get_dict_info(coin_detail1, "platforms")

def get_dict_df(dict_info: dict) -> DataFrame:
    df: DataFrame = pd.DataFrame(dict_info)
    columns: List[str] = df.columns.to_list()
    columns.remove("id")
    ordered_columns: List[str] = ["id", *columns]
    return df[ordered_columns]


dict_df0 = get_dict_df(dict_info0)
dict_df1 = get_dict_df(dict_info1)


def concat_df(df1, df2) -> DataFrame:
    return pd.concat([df1, df2], ignore_index=True)

dict_df = concat_df(dict_df0, dict_df1)

def get_dict_df_all(coins_list: List[dict], column_name: str) -> DataFrame:
    coin_detail_list = [get_coin_details(coin["id"]) for coin in coins_list[:50]]
    dict_info_list = [get_dict_info(coin_detail, column_name) for coin_detail in coin_detail_list]
    dict_df_list = [get_dict_df(dict_info) for dict_info in dict_info_list]
    return reduce(concat_df, dict_df_list)

dict_df_all = get_dict_df_all(coins_list, "platforms")
dict_df_all.head(3)

## Extract Plataforms Data Frame

In [ ]:
dict_location_all = get_dict_df_all(coins_list, "platforms")
dict_location_all.head(3)

## Extract Localization Data Frame

In [ ]:
dict_location_all = get_dict_df_all(coins_list, "localization")
dict_location_all.head(3)